In [1]:
# start by importing the modules we will need
import requests
import json  #we will use these later in retrieving geodata and plotting university locations using openstreetmaps
import folium
import numpy as np
import cufflinks as cf
import pandas as pd #These allow us to open our data in a pandas data frame
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
print("Ready to go to college?".title())

Ready To Go To College?


In [2]:
CENTER_US = (39.8333333,-98.585522)
#london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=4)

In [3]:
import sys
from sys import exit

help(exit)

Help on built-in function exit in module sys:

exit(...)
    exit([status])
    
    Exit the interpreter by raising SystemExit(status).
    If the status is omitted or None, it defaults to zero (i.e., success).
    If the status is an integer, it will be used as the system exit status.
    If it is another kind of object, it will be printed and the system
    exit status will be one (i.e., failure).



In [ ]:
#accepted_states = ['Alabama','Alaska','Arizona','Arkansas' ,'California' ,'Colorado','Connecticut', 'Delaware','Florida','Georgia']
state = input('Enter State: ex. NewYork ')
try:
    state = pd.read_csv(state)
except FileNotFoundError:
    print('Please re-run and follow the correct format. Enter the state in title format. If the state has two words, ignore the space.')
    print("For example: Connecticut, NewYork, NewJersey, etc.")
    try:
        state = input('Enter State: ex. NewYork ')
        state = pd.read_csv(state)
    except FileNotFoundError:
        exit(0)
        print("Follow instructions!")
    

    #while True:
        #state = input('Enter State: ex. NewYork ')
        #if state is not in accepted_states:
            #break
        #else:
            #state = pd.read_csv(state)
#Add column based on size
state['size'] = np.nan
state['size'][state['Student population']>15000] = 'large'
state['size'][state['Student population']<=15000] = 'medium'
state['size'][state['Student population'] <=5000] = 'small'
state

display(state)
#filter by type of institution
college_types = list(state['Type'].value_counts().index)
i = 1
for types in college_types:
    print(i,types)
    i = i + 1
choice = input ('What type of university are you interested in? Enter the correspondong number (1 - 6). If you have no preference, just hit enter. ')
try:
    if choice == '':
        state2 = state
    else:
        choice = int(choice)#convert choice into an integer that corresponds to the menu options
        choice_value = college_types[choice-1]
        state2 = state[state['Type']==choice_value]
    display(state2)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state2 = state

#filter by surrounding area
surrounding_area = list(state2['Campus setting'].value_counts().index)
i = 1
for types in surrounding_area:
    print(i,types)
    i = i + 1
area = input ('What setting would you like the campus to be in? Please choose from the menu by entering the corresponding number or press enter if you have no preference.')
try:
    if area == '':
        state3 = state2
    else:
        area = int(area)
        setting_preference = surrounding_area[area-1]
        state3 = state2[state2['Campus setting']==setting_preference]
    display(state3)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state3=state2

#filter by size
college_sizes = list(state3['size'].value_counts().index)
i = 1
for types in college_sizes:
    print(i,types)
    i = i + 1
print('FYI: Large schools are over 15000. Small schools are less than 5000. Medium schools are in the middle.')
size_preference = input("How big do you want your university to be? Above are your options. Press enter if you have no preference.")
try:
    if size_preference == '':
        state4 = state3
    else:
        size_preference= int(size_preference) 
        size = college_sizes[size_preference -1]
        state4=state3[state3['size']==size]
    display(state4)
except ValueError:
    print('I am assuming you have no preference then. I guess we can move on.')
    state4=state3

#filter by on campus housing
campus_housing = input('Do you want campus housing? Yes or no or type anything you do not have a preference. ')
campus_housing = campus_housing.capitalize()

if campus_housing == 'Yes':
    state5 = state4[state4['Campus housing'] == 'Yes']
    display(state5)
elif campus_housing == 'No':
    state5 = state4[state4['Campus housing'] == 'No']
    display(state5)
else: 
    state5 = state4
    display(state5)    


In [8]:
school_data= pd.DataFrame({'Name':state4['Name'],
              'Location':state4['Address'],'Website': state4['Website']})
display(school_data)
#list(school_data)

#created a new data frame with the information we want. 



def get_coordinates_using_google(location):
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    options = { 'address' : location }  
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = geodata['results'][0]['geometry']['location']
    return coords
try:
    for index, each_school in school_data.iterrows():
        location = each_school['Location']
        coords = get_coordinates_using_google(location)
        name = str(each_school['Name'])
        website = str(each_school['Website'])
        #print(name, website, coords)
        pos = (coords['lat'],coords['lng'])
        message = "%s: %s" % (name,website)
        i=i+1
        marker = folium.CircleMarker(location=pos,
                                    popup=message,
                                    color='#3226cc'
                                    )
        map.add_children(marker)
    display(map)  
except IndexError:
    print("Try running me again.")


,Location,Name,Website
47,"9001 Stockdale Hwy, Bakersfield, California 93...",California State University-Bakersfield,www.csub.edu


In [9]:
i = 0

all_data= pd.DataFrame({'Name':state['Name'],
              'Location':state['Address'],'Website': state['Website']})
def get_coordinates_using_google(location):
    url = 'http://maps.googleapis.com/maps/api/geocode/json'
    options = { 'address' : location }  
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = geodata['results'][0]['geometry']['location']
    return coords
try:
    for index, each_school in all_data.iterrows():
        location = each_school['Location']
        coords = get_coordinates_using_google(location)
        name = str(each_school['Name'])
        website = str(each_school['Website'])
        #print(name, website, coords)
        pos = (coords['lat'],coords['lng'])
        message = "%s: %s" % (name,website)
        i=i+1
        marker = folium.CircleMarker(location=pos,
                                    popup=message,
                                    color='#3226cc'
                                    )
        map.add_children(marker)
    display(map)  
except IndexError:
    print("Stay tuned for next update!")


Stay tuned for next update!
